# Python notebook to show issues with Typing

## General Links

* OpenAI TypeSpec package: https://www.npmjs.com/package/@azure-tools/openai-typespec
* OpenAI TypeSpec repo: https://github.com/microsoft/openai-openapi-pr/tree/main/packages/openai-typespec/src
* Our current OpenAI TypeSpec files: https://github.com/Azure/azure-rest-api-specs-pr/tree/feature/ai-foundry/agents-v2/specification/ai/Azure.AI.Projects/.external-readonly/openai.external.typespec
* OpenAI Python reference: https://platform.openai.com/docs/api-reference 

In [ ]:
# Setup
from dotenv import load_dotenv
load_dotenv()


## Issues in Request payload

### 'model' in ImageGenTool

OpenAI defines model as `Literal['gpt-image-1']` in our current TypeSpec, and `"gpt-image-1" | "gpt-image-1-mini"` in the [published OpenAI TypeSpec package](https://github.com/microsoft/openai-openapi-pr/blob/main/packages/openai-typespec/src/responses/models.tsp#L1742). But we try to pass in any str (with potentially different model name).

In [ ]:
import os
from typing import cast
from azure.ai.projects.models import ImageGenTool

image_generation_model = cast(str, os.getenv("IMAGE_GENERATION_MODEL_DEPLOYMENT_NAME"))

# OpenAI defines model as `Literal['gpt-image-1']` but we try to pass in any str (with potentially different mode name)
tool = ImageGenTool(model=image_generation_model, quality="low", size="1024x1024")

Error:
```
Argument of type "str" cannot be assigned to parameter "model" of type "Literal['gpt-image-1'] | None" in function "__init__"
  Type "str" is not assignable to type "Literal['gpt-image-1'] | None"
    "str" is not assignable to "None"
    "str" is not assignable to type "Literal['gpt-image-1']"Pylance(reportArgumentType)
```

### '.evals.create' Azure extensions

Example taken from sample: samples\evaluations\sample_continuous_evaluation_rule.py
Link to OpenAI Python reference code for .evals.create(): https://platform.openai.com/docs/api-reference/evals/create

In [ ]:
import os
from azure.identity import DefaultAzureCredential
from azure.ai.projects import AIProjectClient

endpoint = os.environ["AZURE_AI_PROJECT_ENDPOINT"]

with (
    DefaultAzureCredential() as credential,
    AIProjectClient(endpoint=endpoint, credential=credential) as project_client,
    project_client.get_openai_client() as openai_client,
):

    data_source_config = {"type": "azure_ai_source", "scenario": "responses"}
    testing_criteria = [
        {"type": "azure_ai_evaluator", "name": "violence_detection", "evaluator_name": "builtin.violence"}
    ]
    eval_object = openai_client.evals.create(
        name="Continuous Evaluation",
        data_source_config=data_source_config,
        testing_criteria=testing_criteria,
    )


Regarding data_source_config:

Error:
```
Argument of type "dict[str, str]" cannot be assigned to parameter "data_source_config" of type "DataSourceConfig" in function "create"
  Type "dict[str, str]" is not assignable to type "DataSourceConfig"
    "dict[str, str]" is not assignable to "DataSourceConfigCustom"
    "dict[str, str]" is not assignable to "DataSourceConfigLogs"
    "dict[str, str]" is not assignable to "DataSourceConfigStoredCompletions"Pylance(reportArgumentType)
```

Analysis:
Per OpenAI ref doc, for data_source_config, there is not support for {"type": "something", "scenario": "something"}. This is an Azure extension. In out TypeSpec we have:

```JavaScript
model AzureAIDataSourceConfig extends DataSourceConfig {
  /** The object type, which is always `azure_ai_source`. */
  type: "azure_ai_source";

  /** Data schema scenario. */
  #suppress "@azure-tools/typespec-azure-core/no-unnamed-union" "Auto-suppressed warnings non-applicable rules during import."
  @added(Versions.v2025_11_15_preview)
  scenario: "red_team" | "responses" | "traces";
}
```

Regarding testing_criteria: 

Error:
```
Argument of type "list[dict[str, str]]" cannot be assigned to parameter "testing_criteria" of type "Iterable[TestingCriterion]" in function "create"
  "list[dict[str, str]]" is not assignable to "Iterable[TestingCriterion]"
    Type parameter "_T_co@Iterable" is covariant, but "dict[str, str]" is not a subtype of "TestingCriterion"
      Type "dict[str, str]" is not assignable to type "TestingCriterion"
        "dict[str, str]" is not assignable to "TestingCriterionLabelModel"
        "dict[str, str]" is not assignable to "StringCheckGraderParam"
        "dict[str, str]" is not assignable to "TestingCriterionTextSimilarity"
        "dict[str, str]" is not assignable to "TestingCriterionPython"
        "dict[str, str]" is not assignable to "TestingCriterionScoreModel"
```

Analysis: 

We extended OpenAI definition by adding the last one below:

```javascript
@@changePropertyType(OpenAI.Eval.testing_criteria,
  (
    | OpenAI.EvalGraderLabelModel
    | OpenAI.EvalGraderStringCheck
    | OpenAI.EvalGraderTextSimilarity
    | OpenAI.EvalGraderPython
    | OpenAI.EvalGraderScoreModel
    | EvalGraderAzureAIEvaluator)[]
);
```

### evals.run.create

samples\evaluations\sample_agent_evaluation.py

In [ ]:
import os
import time
from typing import Union
from pprint import pprint
from dotenv import load_dotenv
from azure.identity import DefaultAzureCredential
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import PromptAgentDefinition
from openai.types.eval_create_params import DataSourceConfigCustom
from openai.types.evals.run_create_response import RunCreateResponse
from openai.types.evals.run_retrieve_response import RunRetrieveResponse

load_dotenv()
endpoint = os.environ["AZURE_AI_PROJECT_ENDPOINT"]

# [START agent_evaluation_basic]
with (
    DefaultAzureCredential() as credential,
    AIProjectClient(endpoint=endpoint, credential=credential) as project_client,
    project_client.get_openai_client() as openai_client,
):
    agent = project_client.agents.create_version(
        agent_name=os.environ["AZURE_AI_AGENT_NAME"],
        definition=PromptAgentDefinition(
            model=os.environ["AZURE_AI_MODEL_DEPLOYMENT_NAME"],
            instructions="You are a helpful assistant that answers general questions",
        ),
    )
    print(f"Agent created (id: {agent.id}, name: {agent.name}, version: {agent.version})")

    data_source_config = DataSourceConfigCustom(
        type="custom",
        item_schema={"type": "object", "properties": {"query": {"type": "string"}}, "required": ["query"]},
        include_sample_schema=True,
    )
    testing_criteria = [
        {
            "type": "azure_ai_evaluator",
            "name": "violence_detection",
            "evaluator_name": "builtin.violence",
            "data_mapping": {"query": "{{item.query}}", "response": "{{item.response}}"},
        }
    ]
    eval_object = openai_client.evals.create(
        name="Agent Evaluation",
        data_source_config=data_source_config,
        testing_criteria=testing_criteria,  # type: ignore
    )
    print(f"Evaluation created (id: {eval_object.id}, name: {eval_object.name})")

    data_source = {
        "type": "azure_ai_target_completions",
        "source": {
            "type": "file_content",
            "content": [
                {"item": {"query": "What is the capital of France?"}},
                {"item": {"query": "How do I reverse a string in Python?"}},
            ],
        },
        "input_messages": {
            "type": "template",
            "template": [
                {"type": "message", "role": "user", "content": {"type": "input_text", "text": "{{item.query}}"}}
            ],
        },
        "target": {
            "type": "azure_ai_agent",
            "name": agent.name,
            "version": agent.version,  # Version is optional. Defaults to latest version if not specified
        },
    }

    agent_eval_run: Union[RunCreateResponse, RunRetrieveResponse] = openai_client.evals.runs.create(
        eval_id=eval_object.id,
        name=f"Evaluation Run for Agent {agent.name}",
        data_source=data_source
    )


Error:
```
Argument of type "dict[str, Unknown]" cannot be assigned to parameter "data_source" of type "DataSource" in function "create"
  Type "dict[str, Unknown]" is not assignable to type "DataSource"
    "dict[str, Unknown]" is not assignable to "CreateEvalJSONLRunDataSourceParam"
    "dict[str, Unknown]" is not assignable to "CreateEvalCompletionsRunDataSourceParam"
    "dict[str, Unknown]" is not assignable to "DataSourceCreateEvalResponsesRunDataSource"
```

data_source "type" "azure_ai_target_completions" is an Azure extension type. In our TypeSpec we have:

```javascript
@@changePropertyType(OpenAI.CreateEvalRunRequest.data_source,

    | OpenAI.CreateEvalJsonlRunDataSource
    | OpenAI.CreateEvalCompletionsRunDataSource
    | OpenAI.CreateEvalResponsesRunDataSource
    | EvalRunDataSource /* RedTeamEvalRunDataSource | TargetCompletionsEvalRunDataSource | AzureAIResponsesEvalRunDataSource | TracesEvalRunDataSource */
);

/** Base class for run data sources with discriminator support. */
@discriminator("type")
model EvalRunDataSource {
  /** The data source type discriminator. */
  type: string;
}

model TargetCompletionsEvalRunDataSource extends EvalRunDataSource {
  /** The type of data source, always `azure_ai_target_completions`. */
  type: "azure_ai_target_completions";

  /** Input messages configuration. */
  input_messages?: OpenAI.CreateEvalCompletionsRunDataSourceInputMessagesItemReference;

  /** The source configuration for inline or file data. */
  #suppress "@azure-tools/typespec-autorest/union-unsupported" "This union is defined according to the Azure OpenAI API."
  #suppress "@azure-tools/typespec-azure-core/no-unnamed-union" "Auto-suppressed warnings non-applicable rules during import."
  source: OpenAI.EvalJsonlFileContentSource | OpenAI.EvalJsonlFileIdSource;

  /** The target configuration for the evaluation. */
  target: Target;
}
```


### Fine Tuning

From sample: samples\finetuning\sample_finetuning_reinforcement_job.py
Link to OpenAI Python reference: https://platform.openai.com/docs/api-reference/fine-tuning/create


In [ ]:
import os
from typing import Any, Dict
from dotenv import load_dotenv
from azure.identity import DefaultAzureCredential
from azure.ai.projects import AIProjectClient
from pathlib import Path

load_dotenv()

endpoint = os.environ["AZURE_AI_PROJECT_ENDPOINT"]
model_name = os.environ.get("MODEL_NAME", "o4-mini")
script_dir = Path(__file__).parent
training_file_path = os.environ.get("TRAINING_FILE_PATH", os.path.join(script_dir, "data", "rft_training_set.jsonl"))
validation_file_path = os.environ.get(
    "VALIDATION_FILE_PATH", os.path.join(script_dir, "data", "rft_validation_set.jsonl")
)

with (
    DefaultAzureCredential() as credential,
    AIProjectClient(endpoint=endpoint, credential=credential) as project_client,
    project_client.get_openai_client() as openai_client,
):

    print("Uploading training file...")
    with open(training_file_path, "rb") as f:
        train_file = openai_client.files.create(file=f, purpose="fine-tune")
    print(f"Uploaded training file with ID: {train_file.id}")

    print("Uploading validation file...")
    with open(validation_file_path, "rb") as f:
        validation_file = openai_client.files.create(file=f, purpose="fine-tune")
    print(f"Uploaded validation file with ID: {validation_file.id}")

    print("Waits for the training and validation files to be processed...")
    openai_client.files.wait_for_processing(train_file.id)
    openai_client.files.wait_for_processing(validation_file.id)

    grader: Dict[str, Any] = {
        "name": "Response Quality Grader",
        "type": "score_model",
        "model": "o3-mini",
        "input": [
            {
                "role": "user",
                "content": "Evaluate the model's response based on correctness and quality. Rate from 0 to 10.",
            }
        ],
        "range": [0.0, 10.0],
    }

    print("Creating reinforcement fine-tuning job")
    fine_tuning_job = openai_client.fine_tuning.jobs.create(
        training_file=train_file.id,
        validation_file=validation_file.id,
        model=model_name,
        method={
            "type": "reinforcement",
            "reinforcement": {
                "grader": grader,
                "hyperparameters": {
                    "n_epochs": 1,
                    "batch_size": 4,
                    "learning_rate_multiplier": 2,
                    "eval_interval": 5,
                    "eval_samples": 2,
                    "reasoning_effort": "medium",
                },
            },
        },
        extra_body={
            "trainingType": "Standard"
        },  # Recommended approach to set trainingType. Omitting this field may lead to unsupported behavior.
    )
    print(fine_tuning_job)


Error:
```
Argument of type "dict[str, str | dict[str, Dict[str, Any] | dict[str, int | str]]]" cannot be assigned to parameter "method" of type "Method | Omit" in function "create"
  Type "dict[str, str | dict[str, Dict[str, Any] | dict[str, int | str]]]" is not assignable to type "Method | Omit"
    "dict[str, str | dict[str, Dict[str, Any] | dict[str, int | str]]]" is not assignable to "Method"
    "dict[str, str | dict[str, Dict[str, Any] | dict[str, int | str]]]" is not assignable to "Omit"
```


Analysis:
Why do we get squiggly lines for "method="? We are not using any Azure extensions here...
method "type" and "reinforcement" are valid fields ("reinforcement" is a recognized value).
reinforcement.grader seems valid, based on OpenAI class `ScoreModelGraderParam`.
reinforcement.hyperparameters seems valid, based on OpenAI class `ReinforcementHyperparametersParam`.

CoPilot said "Generic dictionaries are not compatible with TypedDict types in static type checking, even if the structure matches perfectly at runtime"... So what's the point in using TypedDict?